In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, box, Polygon
import numpy as np
import matplotlib.pyplot as plt
import ast
import datetime

In [2]:
def setupApiFile():
    apiFile = pd.read_csv("../data/processed/crash_street_api_features.csv")

    # remove columns we basically already have
    apiFile.drop(columns={"maxspeed", "road_type"}, inplace=True)

    # just print out before filterin
    print(f"With and without lane count, we have: {apiFile.shape[0]} crashes")

    apiFile = apiFile.dropna(subset=["lane_count"])

    apiFile.fillna(0, inplace=True)

    print(f"With only lane count, we have: {apiFile.shape[0]} crashes")

    for index, crash in apiFile.iterrows():
        try:
            apiFile.at[index, "lane_count"] = int(crash["lane_count"])
        except:
            laneCount = ast.literal_eval(crash["lane_count"])
            laneCount = int(laneCount[0]) + int(laneCount[1])
            apiFile.at[index, "lane_count"] = laneCount
    return apiFile

In [3]:
apiFile = setupApiFile()
crash_frame = pd.read_csv("../data/processed/encoded_data_binary_encoding.csv")

gdf = gpd.GeoDataFrame(
    crash_frame, 
    geometry=gpd.points_from_xy(crash_frame["Longitude"], crash_frame["Latitude"]), 
    crs="EPSG:4326"
)

# left upper bound - 33°15'08.7"N 97°21'46.3"W
# right lower bound - 32°22'36.5"N 96°07'37.0"W

#using our bounds, create the corners we'll use in our polygon
upperLeft = (-1 * (97 + 21/60 + 46.3/3600), 33 + 15/60 + 8.7/3600)
bottomRight = (-1 * (96 + 7/60 + 37/3600), 32 + 22/60 + 36.5/3600)

upperRight = (bottomRight[0], upperLeft[1])
bottomLeft = (upperLeft[0], bottomRight[1])

# create the polygon
dallasBounds = Polygon([upperLeft, upperRight, bottomRight, bottomLeft, upperLeft])

# create teh dataframe
dallasFrame = gpd.GeoDataFrame({"geometry": [dallasBounds]}, crs="EPSG:4326")

# cahnge coordinate system of both
dallasFrame = dallasFrame.to_crs(epsg=32614)
gdf = gdf.to_crs(epsg=32614)

cell_size = 500

minx, miny, maxx, maxy = dallasFrame.total_bounds

grids = []
index = 0
# create grids based on the bounds
for x in np.arange(minx, maxx, cell_size):
    for y in np.arange(miny, maxy, cell_size):
        grids.append(box(x, y, x+cell_size, y+cell_size))

# 2. Optional: intersect with Dallas boundary to crop
grid = gpd.GeoDataFrame({"geometry": grids}, crs=dallasFrame.crs)

grid = gpd.overlay(grid, dallasFrame, how="intersection")

# combine it based on which cells are matching
joined_data = gpd.sjoin(gdf, grid, how="left", predicate="intersects")

# then, join by api data
joined_data = pd.merge(joined_data, apiFile, on=["Latitude", "Longitude"], how="inner")

final_cells = []

cellId = 0

timeRange = 6

# group by when they go to similar cells
for _, crashes in joined_data.groupby("index_right"):
    cellIndex = crashes.iloc[0]["index_right"]
    print(f'In cell {cellIndex} we have {len(crashes)} crashes!')
    crashes.drop(columns=["index_right", "Crash ID", "geometry"], inplace=True)

    crashes["crash_count_7d"] = 0
    crashes["crash_count_30d"] = 0
    crashes["cell_id"] = cellId
    crashes["label"] = 0

    crashes["Crash Date"] = pd.to_datetime(crashes["Crash Date"], format="%Y-%m-%d")
    crashes = crashes.sort_values(["Crash Date", "Hour of Day"], ascending=True).reset_index(drop=True)

    # loop through each crash to add it to each timeslot
    for i, crash in crashes.iterrows():        
        crashIndex = crash["Hour of Day"]
        # loop through each possible timeslot represneted by j, and put into the cells the value with the correct label based on if the time matches.
        for currentLowerHourBound in range(0, 24, 6):
            currentUpperHourBound = currentLowerHourBound + 5

            crashToAppend = crash.copy()
            # check if the timeslot matches. If so, put crash with label of 1, otherwise keep it as the 0. Also, keep current hour of day
            if(crashIndex >= currentLowerHourBound and crashIndex <= currentUpperHourBound):
                crashToAppend["label"] = 1
            # otherwise, keep label as 0, and make the hour of day the halfway point between the 2 bounds
            else:
                crashToAppend["Hour of Day"] = int((currentLowerHourBound + currentUpperHourBound) / 2)

            rolling_7_count = 0
            rolling_30_count = 0
            currentCrashDate = crash["Crash Date"]
            # loop through previous crashes
            for z in range(i - 1, -1, -1):
                prevCrashDate = crashes.iloc[z]["Crash Date"]
                daysPassed = (currentCrashDate - prevCrashDate).days
                # use this if sattement to add it
                if(daysPassed == 0):
                    prevHour = crashes.iloc[z]["Hour of Day"]
                    if(prevHour < crashToAppend["Hour of Day"]):
                        rolling_7_count += 1
                        rolling_30_count += 1
                if(daysPassed > 30):
                    break
                # note that we don't use an else statement. This way if both are true, it'll be added to both
                if(daysPassed <= 7):
                    rolling_7_count += 1
                if(daysPassed <= 30):
                    rolling_30_count += 1
            # now, add it to the count
            crashToAppend["crash_count_7d"] = rolling_7_count
            crashToAppend["crash_count_30d"] = rolling_30_count
            final_cells.append(crashToAppend)
    cellId += 1

With and without lane count, we have: 41632 crashes
With only lane count, we have: 33649 crashes
In cell 12170 we have 27 crashes!
In cell 12178 we have 5 crashes!
In cell 12184 we have 13 crashes!
In cell 12188 we have 61 crashes!
In cell 12206 we have 82 crashes!
In cell 12365 we have 46 crashes!
In cell 12372 we have 1 crashes!
In cell 12373 we have 4 crashes!
In cell 12379 we have 9 crashes!
In cell 12383 we have 167 crashes!
In cell 12384 we have 29 crashes!
In cell 12401 we have 62 crashes!
In cell 12560 we have 129 crashes!
In cell 12567 we have 35 crashes!
In cell 12574 we have 22 crashes!
In cell 12575 we have 5 crashes!
In cell 12578 we have 64 crashes!
In cell 12579 we have 10 crashes!
In cell 12595 we have 15 crashes!
In cell 12596 we have 103 crashes!
In cell 12755 we have 104 crashes!
In cell 12762 we have 39 crashes!
In cell 12770 we have 18 crashes!
In cell 12773 we have 27 crashes!
In cell 12774 we have 1 crashes!
In cell 12790 we have 1 crashes!
In cell 12791 we have 

In [4]:
exportedDf = pd.DataFrame(final_cells)

exportedDf.drop(columns="Crash Date", inplace=True)

exportedDf.to_csv("../data/final/true_preprocessed_data.csv", index=False)